In [57]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [58]:
words = open("names.txt", "r").read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [59]:
# build vocab of characters:
chars = sorted(list(set(''.join(words))))

# create mapping from int to chars and vice-versa
stoi, itos = {}, {}
for num, char in enumerate(chars):
    stoi[char] = num + 1
    itos[num+1] = char

stoi['.'] = 0
itos[0] = '.'

In [60]:
# build the dataset

block_size = 3 # context-definition: how many prev chars to be fed to the net

X, Y = [], [] # list of tensors of ints corresponding to characters 

"""my for loop"""
# for word in words[:3]:
#     for _ in range(block_size):
#         word = '.' + word
#     word += '.'
#     for start_idx in range(0,len(word)-block_size):
#         input_context = ""
#         # star from start_idx and take the next "block_size" chars
#         for a in range(block_size):
#             input_context += word[start_idx + a]
#         X.append(input_context)
#         Y.append(word[start_idx + block_size])
# for idx in range(len(X)):
#     print(X[idx]," ---> ", Y[idx])

"""andrej's for loop"""
for word in words[:5]:
    context = [0] * block_size
    for ch in word + '.':
        Y.append([stoi[ch]]) # this is the label/target
        X.append(context)
        print(''.join(itos[i] for i in context), "-->", ch)
        context = context[1:] + [stoi[ch]]

X = torch.tensor(X)
Y = torch.tensor(Y)

... --> e
..e --> m
.em --> m
emm --> a
mma --> .
... --> o
..o --> l
.ol --> i
oli --> v
liv --> i
ivi --> a
via --> .
... --> a
..a --> v
.av --> a
ava --> .
... --> i
..i --> s
.is --> a
isa --> b
sab --> e
abe --> l
bel --> l
ell --> a
lla --> .
... --> s
..s --> o
.so --> p
sop --> h
oph --> i
phi --> a
hia --> .


In [61]:
# lets cram these 27 chars into 2 dimentional space: (this is our EMBEDDINGS)
# (each of the 27 chars is represented by a 2-dimentional vector)

# look-up table for creating embedddings:
C = torch.randn((27, 2))

In [76]:
# we want embeddings for all the integers in matrix X:
emb = C[X]
emb.shape

# emb is our 1st layer of MLP

torch.Size([32, 3, 2])

In [ ]:
# 2nd layer (hidden)
w1 = torch.randn((6, 100))
b1 = torch.randn(100)

# getting inputs from the 1st layer for the second layer
# METHOD 1: (not flexible with block-size)

# torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape

# METHOD 2:
# we want to unbind the 1st dimention:
ips = torch.unbind(emb, 1) # this is exactly the sequence of tensors passed in METHOD 1
# returns a sequence 
torch.cat(ips, 1).shape

torch.Size([32, 6])

In [74]:
# METHOD 3: (more efficient)

